In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
%reload_ext autoreload
%aimport pdlmfit

In [2]:
data = pd.read_csv('demo/GCN4_fourfield_R2.tsv', sep='\t')
data = data.loc[(data.resi<=10)&(data.field<17.0)].reset_index(drop=True)

In [3]:
def exponential_decay(par, xdata):
    
    # Parse multiple input parameter
    # formats for intensity, rate    
    if hasattr(par,'valuesdict'):
        # lmfit parameter format
        var = par.valuesdict()
        inten = var['inten']
        rate = var['rate']
    elif hasattr(par,'keys'):
        # dict format
        inten = par['inten']
        rate = par['rate']
    else:
        # array/list/tuple format
        inten = par[0]
        rate = par[1]

    # Calculate the y-data from the parameters
    return inten * np.exp(-1*rate*xdata)

In [4]:
func = exponential_decay
groupcols = ['resi', 'field']
xname = 'time'
yname = 'intensity'
yerr = None

method = 'leastsq'
sigma = [0.95, 0.75]
threads = 4

params = [{'name':'inten', 'value':np.asarray(data.groupby(groupcols)[yname].max()), 'vary':True},
          {'name':'rate', 'value':20.0, 'vary':True}]

In [5]:
%autoreload 1
a = paNLS.pdlmfit(data, exponential_decay, groupcols, params, 
                  xname, yname, 
                  method=method, sigma=sigma, threads=threads)
a.fit()

/Volumes/Files/miniconda/envs/scienv2/lib/python2.7/site-packages/pandas/core/indexing.py:1294: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


In [6]:
a.results

inten                                          rate  \
                   value      stderr      ci0.95      ci0.75     value   
resi field                                                               
5    14.1   15313.367271  139.475597  232.712163  133.535301  5.085937   
     16.5   68692.303692  362.676887  618.139941  351.026619  5.510422   
6    14.1   12150.037320  125.957061  210.901190  120.952984  5.913539   
     16.5   67337.150278  336.018198  571.439581  325.449861  5.697662   
7    14.1   14697.948394  167.021450  280.190255  160.521817  6.200861   
     16.5   77261.569261  333.556144  554.809270  318.306887  5.310241   
8    14.1   13840.535301  132.637233  221.768467  127.234246  6.242079   
     16.5   74100.684126  486.329832  835.998981  472.720334  7.753889   
9    14.1   12171.142895  104.751966  175.113088  100.440851  6.830633   
     16.5   64055.258667  322.204238  559.665870  316.124770  7.071610   
10   14.1   10138.093669   81.913608  136.866720   78.516162  7.178645   
     16.5   50868.876634  212.575306  364.942552  206.813049  6.945686   

                                          
              stderr    ci0.95    ci0.75  
resi field                                
5    14.1   0.153085  0.266542  0.150533  
     16.5   0.132718  0.222091  0.128175  
6    14.1   0.227691  0.403341  0.227325  
     16.5   0.130639  0.239723  0.134628  
7    14.1   0.268259  0.479337  0.269351  
     16.5   0.080372  0.130985  0.075911  
8    14.1   0.221234  0.394997  0.221731  
     16.5   0.145327  0.238069  0.137159  
9    14.1   0.215821  0.384843  0.216192  
     16.5   0.094696  0.159837  0.091165  
10   14.1   0.204430  0.361620  0.203726  
     16.5   0.127468  0.200302  0.118117

In [34]:
def predict(self, df, xcalc=False, xtype='global', xnum=50):
    
    # Get the index for selection
    index = df.index.unique()[0]
    
    # Choose the xdata
    xname = self._xname
    xdata = self.data.loc[index, xname]
    
    if xcalc:
        if xtype == 'global':
            xmin = self.data[xname].min()
            xmax = self.data[xname].max()
        else:
            xmin = xdata.min()
            xmax = xdata.max()
            
        xname = 'xdata'
        xdata = pd.Series(np.linspace(xmin, xmax, xnum))
        xindex = pd.Index(np.array([index]*xnum))
    
    # Pick out the parameters for this data
    params = ( self
             .results
             .sortlevel(axis=1)
             .loc[[index], (slice(None),['value'])]
             )
    params.columns = params.columns.levels[0]
    params = params[self._paramnames].squeeze()
    
    
    model_eq = self._fitobj.model_eq.loc[index]
    
    ycalc = model_eq(params, xdata)
    
    if not xcalc:
        return ycalc
    else:
        return pd.DataFrame({xname:xdata, 'ycalc':ycalc})




z = ( a
     .data
     .groupby(level=range(a._ngroupcols), group_keys=False, as_index=False)
     .apply(lambda x: predict(a, x, True, xnum=5))
     )
z

,xdata,ycalc
0,0.004,15004.983475
1,0.055,11576.759543
2,0.106,8931.790011
3,0.157,6891.122901
4,0.208,5316.691814
0,0.004,67194.774141
1,0.055,50732.360278
2,0.106,38303.162892
3,0.157,28919.062301
4,0.208,21834.023647
